# Text Classification Notebook

In [1]:
!pip install --upgrade pip
!pip install tensorflow
!pip install pandas
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
^C


In [211]:
# Machine Learning related imports
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input

In [212]:
# Dataset and generic imports
import pandas as pd
import numpy as np
import os

In [213]:
# Load dataset
filepath = os.getcwd()
datasetpath =  os.path.join(filepath, "cleaned_data", "laporanencoded.csv")

laporan = pd.read_csv(datasetpath)
# Print dataset shape
print(laporan.shape)
# Print dataset head
laporan.head()


(2852, 27)


,text,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,mohon bantuannya untuk menormalkan sistem di l...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ass pak gubsaya perangkat desa karangsari kec ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,selamat sore bapak gubernur atau yang mewakili...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,assalamualaikum pak gub saya pengurus paguyuba...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,lapor saya okada arle sandi email okadaarlegm...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
# Split text and labels
x = laporan["text"]
y = laporan[[
    "perselisihan",
    "infrastruktur",
    "pemerintah",
    "kesehatan",
    "teknologi",
    "administrasi",
    "fasilitas",
    "lingkungan",
    "ketertiban",
    "listrik",
    "bahaya",
    "lainnya",
    "pungli",
    "ilegal",
    "lalulintas",
    "bencana",
    "air",
    "pendidikan",
    "kebersihan",
    "sosial",
    "wisata",
    "sara",
    "pencurian",
    "korupsi",
    "bbm",
    "keuangan"
]]

In [226]:
# Text head
x.head()

0    mohon bantuannya untuk menormalkan sistem di l...
1    ass pak gubsaya perangkat desa karangsari kec ...
2    selamat sore bapak gubernur atau yang mewakili...
3    assalamualaikum pak gub saya pengurus paguyuba...
4    lapor saya okada arle sandi email  okadaarlegm...
Name: text, dtype: object

In [227]:
# Label head
y.head()

,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,listrik,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [228]:
# Tokenize and pad text
vocab_size = 2000
embedding_dim = 16
max_length = 300
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)

padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
# x = tf.data.Dataset.from_tensor_slices(padded)
# y = tf.data.Dataset.from_tensor_slices(y)

In [229]:
# Shuffle, batch and separate data into train, dev and test
# BUFFER_SIZE = 1000
# BATCH_SIZE = 1
# DATASET_SIZE = len(x)

# dataset = tf.data.Dataset.zip((x, y))
# dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# train_set = dataset.take(int(0.8 * DATASET_SIZE))
# test_set = dataset.skip(int(0.8 * DATASET_SIZE))
# test_set = dataset.take(int(0.2 * DATASET_SIZE)) 

# print(len(train_set))
# print(len(test_set))

In [230]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, None, 64)          128000    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 128)               66048     
_________________________________________________________________
dense_51 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_52 (Dense)             (None, 26)                1690      
Total params: 203,994
Trainable params: 203,994
Non-trainable params: 0
_________________________________________________________________


In [231]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [232]:
NUM_EPOCHS = 5
history = model.fit(padded, y, epochs=NUM_EPOCHS)

Epoch 1/5
90/90 [==============================] - 78s 552ms/step - loss: 0.2377 - accuracy: 0.2300
Epoch 2/5
90/90 [==============================] - 60s 671ms/step - loss: 0.1266 - accuracy: 0.2430
Epoch 3/5
90/90 [==============================] - 52s 584ms/step - loss: 0.1263 - accuracy: 0.2430
Epoch 4/5
90/90 [==============================] - 53s 594ms/step - loss: 0.1265 - accuracy: 0.2430
Epoch 5/5
90/90 [==============================] - 51s 572ms/step - loss: 0.1260 - accuracy: 0.2430
